 # SQL

In this task a SQLite database is used. It is included in the reposiotory.

### Import libraries and set variables

In [2]:
import sqlite3
from tabulate import tabulate

database_file = "../sql/sales.sqlite3"

### Create the database and insert the data

#### Table **transactions**

In [3]:
conn = sqlite3.connect(database_file)

query = 'DROP TABLE IF EXISTS transactions;'
conn.execute(query)

query = 'CREATE TABLE transactions ("id" integer,"date" text,"order_id" integer,"client_id" integer,"prod_price" real,"prod_qty" integer, "prod_id" integer, PRIMARY KEY ("id"));'
conn.execute(query)

query = '''INSERT INTO transactions ("id", "date", "order_id", "client_id", "prod_price", "prod_qty", "prod_id") 
            VALUES
                ('1', STRFTIME('%d/%m/%Y', '2020-01-01'), '1234', '999', '3.56', '4', '389728'),
                ('2', STRFTIME('%d/%m/%Y', '2020-01-01'), '1234', '999', '50', '1', '490756'),
                ('3', STRFTIME('%d/%m/%Y', '2020-01-01'), '3456', '845', '50', '2', '490756'),
                ('4', STRFTIME('%d/%m/%Y', '2020-01-01'), '3456', '845', '300', '1', '549380'),
                ('5', STRFTIME('%d/%m/%Y', '2020-01-01'), '3456', '845', '10', '6', '293718'),
                ('6', STRFTIME('%d/%m/%Y', '2020-03-01'), '1234', '999', '50', '1', '490756'),
                ('7', STRFTIME('%d/%m/%Y', '2020-03-01'), '3456', '845', '50', '2', '490756');'''
conn.execute(query)

conn.commit()
conn.close()

#### Table **product_nomenclature**

In [4]:
conn = sqlite3.connect(database_file)

query = 'DROP TABLE IF EXISTS product_nomenclature;'
cursor = conn.execute(query)

query = 'CREATE TABLE product_nomenclature ("id" integer, "product_type" varchar FIRST, "product_name" varchar FIRST, "product_id" integer FIRST, PRIMARY KEY ("id"));'
cursor = conn.execute(query)

query = '''INSERT INTO product_nomenclature ("id", "product_type", "product_name", "product_id") VALUES
('1', 'DECO', 'Mug', '293718'),
('2', 'DECO', 'Boule de Noël', '389728'),
('3', 'MEUBLE', 'Chaise', '490756'),
('4', 'MEUBLE', 'Canapé', '549380');'''
cursor = conn.execute(query)

conn.commit()
conn.close()

### Task 1.
Start by making a simple SQL query to __find the turnover__ (the total amount of sales), __day by day__, from 1 January 2019 to 31 December 2019. 

The result will be sorted on the date the order was placed.

The query must be clear: don't hesitate to use AS keywords to name the fields in SQL.

In [5]:
conn = sqlite3.connect(database_file)

# The conversion of Y/m/d to Y-m-d with strftime() is in this case specific for SQLite, which is the database used for this demo.
query = '''
        SELECT strftime("%Y-%m-%d",datetime(substr(date, 7, 4) || "-" || substr(date, 4, 2) || "-" || substr(date, 1, 2))) AS date, 
        SUM(prod_price * prod_qty) AS ventes
        FROM transactions
        WHERE date BETWEEN "01/01/2020" AND "31/12/2020"
        GROUP BY date;'''

cursor = conn.execute(query)
results = cursor.fetchall()
conn.close()

print(tabulate(results, headers=["date", "ventes"]))


date          ventes
----------  --------
2020-01-01    524.24
2020-03-01    150


### Task 2.
Carry out a slightly more complex query that makes it possible to __determine, by customer__ and over the period from 1 January 2019 to 31 December 2019, __the furniture and decoration sales__ made.

In [6]:
conn = sqlite3.connect(database_file)

query = '''
        SELECT transactions.client_id AS client_id,
            SUM(CASE
                    WHEN product_nomenclature.product_type = "MEUBLE" 
                    THEN transactions.prod_price * transactions.prod_qty
                    ELSE 0
                END) AS ventes_meubles,
            SUM(CASE
                    WHEN product_nomenclature.product_type = "DECO" 
                    THEN transactions.prod_price * transactions.prod_qty
                    ELSE 0
                END) AS ventes_deco
        FROM transactions
        JOIN product_nomenclature ON transactions.prod_id = product_nomenclature.product_id
        WHERE transactions.date BETWEEN "01/01/2020" AND "31/12/2020"
        GROUP BY client_id
        ORDER BY client_id;'''

cursor = conn.execute(query)
results = cursor.fetchall()
conn.close()

print(tabulate(results, headers=["client_id", "ventes_meubles", "ventes_deco"]))

  client_id    ventes_meubles    ventes_deco
-----------  ----------------  -------------
        845               500          60
        999               100          14.24
